# CSI-EC - TP1 - Exercice 1 - Receiver

*Warning:* Before running this playbook make sure the following packages are installed.

In [1]:
%pip install ascon
%pip install asyncio

import ascon
import asyncio

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


To start we first need to get the seed for generating the key and nonce that will be used by ascon. This should be the same as the ones provided to the emmiter.

In [2]:
key_seed=input("Seed for key > ")
nonce_seed=input("Seed for nonce > ")

After we have the input parameters we need to generate the key and nonce using ascon in XOF mode. 
Additionally, in this step, we configure some extra parameters such as the hash length and the associated data.

In [3]:
hashlength=16
key=ascon.hash(key_seed.encode(),variant="Ascon-Xof", hashlength=hashlength)
nonce=ascon.hash(nonce_seed.encode(),variant="Ascon-Xof", hashlength=hashlength)
associated_data=f'''message_{len(key_seed)*len(nonce_seed)}'''.encode()

We can now verify that the key and nonce is now hashed.

In [4]:
print(f'''Key: {key}
Nonce: {nonce}''')

Key: b'\xf1\x94\xac_\xfdj\xdc-\xd9\xd5D*\x87\xb6\x8en'
Nonce: b'n\x1d\x8d\xe4\x82j\x04]>5\xac\xc7\r^Y\xff'


Having the needed variables we can now create a function that will reverse the cipher and reveal the messages according to the key, nonce and associated data defined previously.

In [5]:
def read_message(text,key, nonce, associated_data):
    try:
        out_message=ascon.decrypt(key, nonce, associated_data, text, variant="Ascon-128")
    except Exception as e:
        pass
    return out_message

To execute the program all is left is connecting to the server and receive its messages.

In [6]:
on=False
try:
    reader, writer = await asyncio.open_connection('localhost', 8888)
    on=True
except:
    print("Server not found!")

try:
    while on:
        data = await reader.read(256)
        if not data:
            break
        try:
            message = read_message(data, key, nonce, associated_data)
            if message.decode().lower()=='exit':
                 on=False
            else:
                print(f"[Server] {message.decode()}")
            
        except Exception as e:
            print(f"Error decrypting message: {e}")
            continue
except asyncio.CancelledError:
    print("Connection Canceled")


[Server] now i can
